# Some example demographies

We illustrate some advanced options of `momi.make_demography()`.

In [1]:
from momi import make_demography, to_ms_cmd

### Specifying demographies in different time scales

Users can choose to input demographies in ms time scale, the more 'standard' coalescent time scale, or in per-generation units.

In [2]:
# a simple Demography, in ms units
events = [('-eg', 0.0, 'a', 1.0), ('-ej', .5, 'a', 'b'), ('-en', 1., 'b', .5)]
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3))

print "Demography in ms syntax:"
print to_ms_cmd(demo)

Demography in ms syntax:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [3]:
# the same Demography, but specified in the 'standard' (not 'ms') time scale
# this means that there are 2*N_ref generations per unit time (instead of 4*N_ref as in ms)

# notice times are 2x longer in the 'standard' units.
# as consequence, growth rate per unit time is 2x slower
events = [('-eg', 0.0, 'a', 0.5), ('-ej', 1., 'a', 'b'), ('-en', 2., 'b', .5)]

# use time_scale='standard' to specify standard coalescent units
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3), time_scale='standard')

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [4]:
# the same Demography, but specified in per-generation units
# we'll assume a diploid population size of 10,000

# join event 5e3 generations ago. population size drops to 5000 at 1e4 generations ago.
# growth rate per generation is 1e-4 in population 'a'
events = [('-eg',0.0,'a',1e-4),('-ej', 5e3, 'a', 'b'), ('-en', 1e4, 'b', 5000)]

# use default_N to set the diploid population size
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3), default_N=10000)

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


### Ghost populations

A 'ghost' population is one that is not present in the sample.

`momi` syntax is more flexible than `ms` about ghost populations. In particular, ghost populations can be immediately affected by demographic events.

By contrast, in `ms`, ghost populations cannot be modified by events until they have been "created" with the **-es** flag.

In [5]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0), # can set growth rate of 'z' at t=0, even tho its a ghost population
          ('-ep', .5, 'a', 'z', .25), # pulse migration from z to a
          ('-ej', 1.0, 'z', 'a')]

demo = make_demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the growth event (-eg) doesn't happen until the admixture event (-es)

Demography in ms syntax.
-I 1 10 -es 0.5 1 0.75 -en 0.5 2 0.606530659713 -eg 0.5 2 1.0 -ej 1.0 2 1


In `momi`, we can also have sampled populations merge **-ej** with ghost populations, even if there is no admixture event.

In [6]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0),
          ('-ej', .5, 'a', 'z'), # 'z' is empty, so this is equivalent to just changing size/growth of 'a'
          ('-en', 1.0, 'z', 1.0)]

demo = make_demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the -ej event doesn't appear; instead just the size and growth rate of 'a' change

Demography in ms syntax.
-I 1 10 -en 0.5 1 0.606530659713 -eg 0.5 1 1.0 -en 1.0 1 1.0


### Ancient DNA

Use the `sampled_t` parameter to indicate that a population was sampled in the past.

In [7]:
events = [('-ej', 1.0, 'a', 'b')]

# the samples from 'b' are ancient DNA, from time t=0.5
demo = make_demography(events, ['a','b'], (6, 10), sampled_t=[0.0, .5])

try:
    print to_ms_cmd(demo)
except:
    print "Cannot convert demography with archaic populations to ms syntax"

Cannot convert demography with archaic populations to ms syntax


### Demography with a lot of parameters

An example on how to programatically construct a demography with a lot of parameters (too many to conveniently write down by hand).

In [8]:
# piecewise constant demography, with 50 pieces

import random

n_pieces = 50
t = 0.0
events = []
for i in range(n_pieces):
    t = t + random.expovariate(100.)
    N = random.expovariate(1.0)
    events.append(('-en', t, 0, N))

demo = make_demography(events, [0], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)

Demography in ms syntax.
-I 1 10 -en 0.00561488208703 1 0.249831142319 -en 0.00621598067893 1 0.536146380531 -en 0.0147268041875 1 4.72741401841 -en 0.0275197696356 1 2.14288591895 -en 0.036805556167 1 2.34879722957 -en 0.0617668597843 1 0.305870346402 -en 0.0625163465379 1 3.14436647213 -en 0.0657254797332 1 0.450702954442 -en 0.0875663868813 1 1.96284858172 -en 0.093181526492 1 0.739522899509 -en 0.0990567733856 1 2.22175789052 -en 0.101421047573 1 0.633790299313 -en 0.105437659289 1 0.30767392487 -en 0.117514624521 1 0.150619434229 -en 0.120686469177 1 0.373443971913 -en 0.123009871596 1 3.69819680125 -en 0.12819649983 1 2.15055446265 -en 0.13148871216 1 1.03464653302 -en 0.13257892525 1 0.227519876256 -en 0.138619738021 1 0.832101106502 -en 0.141452472254 1 0.816876207751 -en 0.161006971944 1 1.56863423631 -en 0.162805409198 1 0.0902280597725 -en 0.164752821854 1 0.0908567844257 -en 0.19106635023 1 0.512630513745 -en 0.203884012464 1 2.0617130402 -en 0.207516730037 1 0.253306550334